In [115]:
df=pd.read_csv("/content/drive/MyDrive/Data/final_test.csv")

In [116]:
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.tree import export_text

In [117]:
df

,weight,height,age,size
0,62,172.72,28.0,5
1,59,167.64,36.0,4
2,61,165.10,34.0,3
3,65,175.26,27.0,4
4,62,172.72,45.0,3
...,...,...,...,...
119729,63,175.26,42.0,3
119730,45,154.94,29.0,2
119731,61,172.72,31.0,3
119732,74,167.64,31.0,5


In [118]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [119]:
features=df.columns[:-1]
features

Index(['weight', 'height', 'age'], dtype='object')

In [120]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119734 entries, 0 to 119733
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   weight  119734 non-null  int64  
 1   height  119404 non-null  float64
 2   age     119477 non-null  float64
 3   size    119734 non-null  object 
dtypes: float64(2), int64(1), object(1)
memory usage: 3.7+ MB


In [122]:
pd.isna(df['height'])

0         False
1         False
2         False
3         False
4         False
          ...  
119729    False
119730    False
119731    False
119732    False
119733    False
Name: height, Length: 119734, dtype: bool

In [123]:
list(pd.isna(df['height'])).count(True)

330

In [124]:
complete_df = df[~pd.isna(df).any(axis=1)].reset_index(drop=True)
complete_df.head()
complete_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119153 entries, 0 to 119152
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   weight  119153 non-null  int64  
 1   height  119153 non-null  float64
 2   age     119153 non-null  float64
 3   size    119153 non-null  object 
dtypes: float64(2), int64(1), object(1)
memory usage: 3.6+ MB


In [125]:
treatment = {
    'weight': 0.0,
    'age': round(np.nanmean(df['age']), 2),
    'height': round(np.nanmean(df['height']), 2)
}


In [126]:
treated_df = df.fillna(value=treatment)
treated_df.head(10)


,weight,height,age,size
0,62,172.72,28.0,5
1,59,167.64,36.0,4
2,61,165.10,34.0,3
3,65,175.26,27.0,4
4,62,172.72,45.0,3
5,50,160.02,27.0,2
6,53,160.02,65.0,3
7,51,160.02,33.0,1
8,54,167.64,26.0,3
9,53,165.10,32.0,2


In [127]:
from sklearn.model_selection import train_test_split
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(treated_df[features].values, treated_df["size"], test_size=0.1) # 80% training and 20% test
y_test

96548     4
105895    5
83142     3
83370     4
54825     1
         ..
98365     3
49309     3
23500     5
32178     4
93294     1
Name: size, Length: 11974, dtype: object

In [128]:
# Xay dung cay
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

my_tree = DecisionTreeClassifier(splitter='random')
my_tree.fit(X_train, y_train)

DecisionTreeClassifier(splitter='random')

In [129]:
y_pred = my_tree.predict(X_test)

cm = confusion_matrix(y_test, y_pred)

print(cm)

[[ 443  464   99    4    0    4    0]
 [ 311 1190  627   40   26    7    0]
 [  69  677 1678  348  159   26    0]
 [  10   90  709  444  409   74    0]
 [   4   40  419  381  773  328    0]
 [   3   12   67   77  393 1562    1]
 [   0    0    0    1    1    4    0]]


In [130]:
Du_lieu=[]
for i in features:
    x=float(input(str(i)+":"))
    Du_lieu.append(x)
x_pred_sample1 = np.array(Du_lieu).reshape(1,-1)
t = my_tree.predict(x_pred_sample1)
if t == "1": 
    decision ="XXS" 
if t =="2":
    decision ="S" 
if t =="3":
    decision ="L" 
if t =="4": 
    decision ="M" 
if t =="5":
    decision ="X"  
if t =="6":
    decision ="XXL"  
print(f"Điểm dữ liệu ở vị trí {x_pred_sample1} Thuộc lớp {decision}")

weight:20
height:140
age:20
Điểm dữ liệu ở vị trí [[ 20. 140.  20.]] Thuộc lớp XXS


In [131]:
def entropy(target_col):
    """
    Calculate the entropy of a dataset.
    The only parameter of this function is the target_col parameter which specifies the target column
    """
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

def InfoGain(data,split_attribute_name,target_name="class"):
    """
    Calculate the information gain of a dataset. This function takes three parameters:
    1. data = The dataset for whose feature the IG should be calculated
    2. split_attribute_name = the name of the feature for which the information gain should be calculated
    3. target_name = the name of the target feature. The default for this example is "class"
    """    
    #Calculate the entropy of the total dataset
    total_entropy = entropy(data[target_name])
    
    ##Calculate the entropy of the dataset
    
    #Calculate the values and the corresponding counts for the split attribute 
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    
    #Calculate the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    
    #Calculate the information gain
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain
       
###################

###################
def ID3(data,originaldata,features,target_attribute_name="class",parent_node_class = None):
    """
    ID3 Algorithm: This function takes five paramters:
    1. data = the data for which the ID3 algorithm should be run --> In the first run this equals the total dataset
 
    2. originaldata = This is the original dataset needed to calculate the mode target feature value of the original dataset
    in the case the dataset delivered by the first parameter is empty

    3. features = the feature space of the dataset . This is needed for the recursive call since during the tree growing process
    we have to remove features from our dataset --> Splitting at each node

    4. target_attribute_name = the name of the target attribute

    5. parent_node_class = This is the value or class of the mode target feature value of the parent node for a specific node. This is 
    also needed for the recursive call since if the splitting leads to a situation that there are no more features left in the feature
    space, we want to return the mode target feature value of the direct parent node.
    """   
    #Define the stopping criteria --> If one of this is satisfied, we want to return a leaf node#
    
    #If all target_values have the same value, return this value
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    
    #If the dataset is empty, return the mode target feature value in the original dataset
    elif len(data)==0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
    
    #If the feature space is empty, return the mode target feature value of the direct parent node --> Note that
    #the direct parent node is that node which has called the current run of the ID3 algorithm and hence
    #the mode target feature value is stored in the parent_node_class variable.
    
    elif len(features) ==0:
        return parent_node_class
    
    #If none of the above holds true, grow the tree!
    
    else:
        #Set the default value for this node --> The mode target feature value of the current node
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        
        #Select the feature which best splits the dataset
        item_values = [InfoGain(data,feature,target_attribute_name) for feature in features] #Return the information gain values for the features in the dataset
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        
        #Create the tree structure. The root gets the name of the feature (best_feature) with the maximum information
        #gain in the first run
        tree = {best_feature:{}}
        
        
        #Remove the feature with the best inforamtion gain from the feature space
        features = [i for i in features if i != best_feature]
        
        #Grow a branch under the root node for each possible value of the root node feature
        
        for value in np.unique(data[best_feature]):
           # value = value
            #Split the dataset along the value of the feature with the largest information gain and therwith create sub_datasets
            sub_data = data.where(data[best_feature] == value).dropna()
            
            #Call the ID3 algorithm for each of those sub_datasets with the new parameters --> Here the recursion comes in!
            subtree = ID3(sub_data,dataset,features,target_attribute_name,parent_node_class)
            
            #Add the sub tree, grown from the sub_dataset to the tree under the root node
            tree[best_feature][value] = subtree
            
        return(tree)    
                
###################

###################
#Hàm dự đoán
#Dữ liệu dự đoán là 
def predict(query,tree,default = 1):
       
    #1.
    for key in list(query.keys()):
        if key in list(tree.keys()):
            #2.
            try:
                result = tree[key][query[key]] 
            except:
                return default
  
            #3.
            result = tree[key][query[key]]
            #4.
            if isinstance(result,dict):
                return predict(query,result)

            else:
                return result

        
        
"""
Check the accuracy of our prediction.
The train_test_split function takes the dataset as parameter which should be divided into
a training and a testing set. The test function takes two parameters, which are the testing data as well as the tree model.
"""
###################

###################
def train_test_split(dataset):
    training_data = dataset.iloc[:170].reset_index(drop=True)#We drop the index respectively relabel the index
    #starting form 0, because we do not want to run into errors regarding the row labels / indexes
    testing_data = dataset.iloc[170:].reset_index(drop=True)
    return training_data,testing_data
#dataset=dataset.drop('animal_name',axis=1)
def test(data,tree):
    #Create new query instances by simply removing the target feature column from the original dataset and 
    #convert it to a dictionary
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    
    #Create a empty DataFrame in whose columns the prediction of the tree are stored
    predicted = pd.DataFrame(columns=["predicted"]) 
    
    #Calculate the prediction accuracy
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0) 
    print('The prediction accuracy is: ',(np.sum(predicted["predicted"] == data["class"])/len(data))*100,'%')
    
    """
Train the tree, Print the tree and predict the accuracy
"""
#dataset=pd.read_csv("./resources/weather_tv.csv")
#training_data=dataset

    